In [1]:
import numpy as np
import pandas as pd
import random
import sklearn

In [2]:
import sys
sys.path.append('..')


In [5]:
from feature_extract import *
from utils import load_data
from models.lgb_model import lgb_model_for_upload,lgb_model_for_offline_test

In [6]:
import os
path0=os.path.abspath(os.path.dirname(os.getcwd()))
path = path0 +'/data/info_dataset/'
path1 = path0+'/data/'

In [7]:
# get feature

In [8]:
person =person_feature(path)
intent_table = person_intent(path)
work_history, work_history_table = work_history(path)
cert_table = certificate(path)
project_history, project_history_table = project_history(path)
job_table = job_table(path)

In [9]:
# load data
train_data, valid_data, test_data =load_data(path1)
del train_data['Unnamed: 0']
del valid_data['Unnamed: 0']
del test_data['Unnamed: 0']

In [10]:
data_all = pd.concat([train_data,valid_data,test_data], ignore_index=True)
candidate_data, job_data = diliver_current(data_all)

## offline

In [11]:
train=get_k_fold_data(train_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid_data0 =data_merge(valid_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid = get_data(valid_data0, train)


In [10]:
from match_func import *

In [12]:
person.head()

,求职者编号,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,当前工作所在地,语言能力,专业特长
0,33291,1,15,3.0,计算机应用技术,37,网络管理/信息安全管理,NaN,深圳市,普通话：精通 广东话：精通******语：听：良好，说：良好，读：良好，写：,"1、精通计算机软、硬件及网络维护，能迅速处理各种突发的计算机故障 2、熟悉局域网建设和管理,..."
1,2985277,1,12,3.0,计算机应用技术,35,*公关/营销/业务类,文化体育行业,深圳市,普通话：精通 广东话：精通************语：听：精通，说：精通，读,..
2,2982066,0,10,3.0,金融学含保险学,32,出纳,医药销售行业,南山区,普通话：精通 广东话：精通************语：听：精通，说：良好，读,****-****年获校三等奖学金****-****年获单项奖学金
3,3010866,1,10,1.0,物理电子学,34,营销代表/销售顾问,珠宝玉石行业,深圳市,普通话：精通 广东话：良好************语：听：一般，说：不会，读,"我对塑胶厂里的运作流程非常熟悉,从业务,计划,生产,人事,采购,客户服务都有经验."
4,316816964,0,15,1.0,学前教育学,34,小学教育/幼儿教育/保育,行业组织,福田区,NaN,1：拥有丰富的办公室工作经历，能独挡一面处理工作中的相关事情。2：熟练掌握Windows办公...


In [13]:
import time
for func in tqdm(["simhash", "sim_jaccard", "sim_edit", "difflibs"]):
    st =time.time()

    person[f"应聘者专业_最近工作岗位_{func}"]  = person.apply(lambda x: eval(func)(x['应聘者专业'], x['最近工作岗位']), axis=1)
    st2 =time.time()
    df_temp1 = person.groupby("求职者编号")[f"应聘者专业_最近工作岗位_{func}"].agg(
                    [(f"应聘者专业_最近工作岗位_{func}_mean", "mean"),
                     (f"应聘者专业_最近工作岗位_{func}_max", "max"),
                     (f"应聘者专业_最近工作岗位_{func}_min", "min"),
                     (f"应聘者专业_最近工作岗位_{func}_std", "std")]
                ).reset_index()
    print(f"应聘者专业_最近工作岗位_{func}_groupby，共耗时:",time.time() - st)


 25%|█████████████████████                                                               | 1/4 [00:22<01:07, 22.54s/it]

应聘者专业_最近工作岗位_simhash_groupby，共耗时: 22.5424587726593



 50%|██████████████████████████████████████████                                          | 2/4 [00:26<00:34, 17.11s/it]

应聘者专业_最近工作岗位_sim_jaccard_groupby，共耗时: 4.415237188339233



 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:44<00:17, 17.35s/it]

应聘者专业_最近工作岗位_sim_edit_groupby，共耗时: 17.906723022460938



100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:50<00:00, 12.59s/it]

应聘者专业_最近工作岗位_difflibs_groupby，共耗时: 5.478602409362793


In [10]:
train_data.groupby("岗位编号").aggregate({"标签": "mean"}).reset_index()

,岗位编号,标签
0,135144,0.6
1,137045,0.0
2,146798,0.0
3,436321,0.0
4,462715,0.5
...,...,...
1838,44274688,1.0
1839,44285651,1.0
1840,44298679,1.0
1841,44310665,1.0


In [12]:
test_data0 =data_merge(test_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
test = get_data(test_data0, train)
temp_data= pd.concat([train,valid, test], ignore_index=True)

In [15]:
def deliver_record_feature(data_feature):
    for f1, f2 in tqdm([["岗位编号", "求职意向岗位类别"], ["岗位编号", "应聘者专业"], ["岗位编号", "求职意向工作地点"]]):
        data_feature[f"{f1}_{f2}_nunique"] = data_feature.groupby(
            [f1])[f2].transform("nunique")
    # 最大值、最小值、均值、标准差、众数、众数占比
    for f1, f2 in tqdm([["岗位编号", "年龄"], ["岗位编号", "工作年限"], ["岗位编号", "工作经历数"], ["岗位编号", "项目经验数"], ["岗位编号", "总主要业绩字数"],
                        ["岗位编号", "最高学历"], ["岗位编号", "可到职天数"]]):
        df_temp1 = data_feature.groupby(f1)[f2].agg(
            [(f"{f1}_{f2}_mean", "mean"),
             (f"{f1}_{f2}_max", "max"),
             (f"{f1}_{f2}_min", "min"),
             (f"{f1}_{f2}_std", "std"),
             (f"{f1}_{f2}_median", np.median),
             (f"{f1}_{f2}_mode", mode),
             (f"{f1}_{f2}_mode_rate", mode_rate)]
        ).reset_index()
        data_feature = data_feature.merge(df_temp1, how="left")

    # 男性占比、女性个数占比、工作地点符合否占比。
    for f1, f2 in [["岗位编号", "性别"], ["岗位编号", "工作地点符合否"]]:  # 性别众数及占比，工作地点符合及占比
        df_temp2 = data_feature.groupby(f1)[f2].agg(
            [
                (f"{f1}_{f2}_mode", mode),
                (f"{f1}_{f2}_mode_rate", mode_rate)]
        ).reset_index()
        data_feature = data_feature.merge(df_temp2, how="left")
    return data_feature

In [11]:
temp_data=deliver_record_feature(temp_data)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.40s/it]


In [17]:
temp_data[:1].values

array([[775040, 5969825, 0, 1, 12, 3.0, None, 34, '品质/质检', nan, '深圳市',
        '外语语种： 语    外语水平： 一般 ************ ******',
        '1.对五金的C机加,冲压、数冲/折弯/焊接/研磨、拉丝/涂装/装配，等各制程的工艺流程，品质的管理和改善提高熟悉。2.熟悉塑胶的成型/二次加工，对其品质管理,分析改善有一定的经验。3.熟悉光电类的光纤连接线和适配器的生产工艺和质量的管控熟悉。 4.现场的7S管理，员工的培训/考核以及执行力与凝聚力的加强有一定的经验。5.熟悉ISO****质量管理体系和相关品质管理工具,如QC七大手法, 8D等。6.能独立编写品质手册以及相关程序文件，能制做相关多媒体科教资料。7.有组织内审，接洽顾客审厂、第三方审核的经验。',
        2.0, '社会无职', 'IPQC/组长', nan, 2.0, '深圳市', 0,
        '熟悉电脑办公软件运用及多项品质管理手法，认识或了解变压器制程及工艺要求，具有强烈的品质观念。', 46.0, False,
        '一直以来，以以身作则，不断进取的心态去面对工作和生活，在带给别人快乐和收获的同时，也陶冶，充实自己。', '品质质检',
        '深圳市', '其它', 7.0, '品质工程师', 50.0, nan, 4.0, 161.25, 645.0, 8, 88,
        0.5303030303030303, 1.0, 0.4545454545454546, 0.5606060606060606,
        0.5151515151515151, 0.5076923076923077, 0.5303030303030303, 1.0,
        0.4545454545454546, 0.6363636363636364, 0.4696969696969697,
        0.5909090909090908, 0.5606060606060606, 0.5151515151515151,
        0.5076923076923077, 0.4545454545454546, 0.5, 0.4615384

In [12]:
temp_data.shape

(106065, 113)

In [20]:
pd.DataFrame(list(temp_data.columns)).to_csv(r'columns.csv')

In [13]:
from sklearn.preprocessing import LabelEncoder

for f in temp_data.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data[f] = le.fit_transform(temp_data[f].astype('str'))
temp_data["招聘对象"] =le.fit_transform(temp_data["招聘对象"].astype('str'))
temp_data["最近工作岗位"] =le.fit_transform(temp_data["最近工作岗位"].astype('str'))
temp_data["求职意向岗位类别"] =le.fit_transform(temp_data["求职意向岗位类别"].astype('str'))

应聘者专业
最近工作岗位
最近所在行业
当前工作所在地
语言能力
专业特长
招聘职位
对应聘者的专业要求
岗位工作地点
具体要求
自荐信
求职意向工作地点
求职意向所在行业
其他说明
汉语语言能力
英语语言能力
粤语语言能力
日语语言能力


In [30]:
# onehot
data_all_feature=feature_process(temp_data)

In [14]:
# none one-hot
data_all_feature=temp_data

In [25]:
data_all_feature.head(1)

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364


In [21]:
train.shape,valid.shape,test.shape

((28232, 120), (7059, 120), (70774, 120))

In [15]:
train0 = data_all_feature[:train.shape[0]]#.标签.value_counts()
valid0 = data_all_feature[train.shape[0]:train.shape[0]+valid.shape[0]]
test0 = data_all_feature[train.shape[0]+valid.shape[0]:]
train_label = train0[["标签"]]
valid_label = valid0[["标签"]]
train=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]
valid=valid0[list(train0.columns)[:2]+list(train0.columns)[3:]]


In [15]:
train.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,应聘者专业平均被匹配数,年龄平均被匹配数,可到职天数平均被匹配数,项目经验数平均被匹配数,工作经历数平均被匹配数,工作地点符合否平均被匹配数,工作年限是否符合平均被匹配数,岗位工作地点平均被匹配数,求职意向工作地点平均被匹配数,求职意向岗位类别平均被匹配数
0,775040,5969825,0,1,12,3.0,NaN,34,品质/质检,NaN,...,NaN,0.185185,0.146529,NaN,0.192942,0.231023,0.159003,0.257207,0.147321,0.113360
1,799873,6139430,0,1,12,4.0,机电一体化,37,机械设计,零售百货行业,...,0.117794,0.189542,0.146529,NaN,0.151583,0.231023,0.159003,0.257207,0.147321,0.242424
2,779698,6083324,0,1,0,1.0,特殊教育学,31,仓储管理,NaN,...,0.166667,0.137206,0.146529,NaN,0.151583,0.125512,0.155056,0.099193,0.147321,0.044118
3,771109,5873085,0,0,6,4.0,财务管理,33,NaN,NaN,...,0.069444,0.163450,0.146529,NaN,0.143004,0.125512,0.159003,0.094819,0.147321,0.164882
4,823833,6218282,0,0,0,3.0,英语,29,客户服务,NaN,...,0.267358,0.141197,0.146529,NaN,0.175149,0.125512,0.155056,0.099193,0.147321,0.212389


# lgb

In [132]:
import lightgbm
from sklearn.metrics import f1_score
params = {"objective": "binary",
          "learning_rate": 0.05,
          "max_depth": 6,
          "num_leaves": 32,
          "verbose": -1,
          "bagging_fraction": 0.8,
          "feature_fraction": 0.9,
          'subsample':0.85,
          'bagging_freq':1,
          'random_state':2048}
def lgb_model_for_offline_test(train, train_label, valid, valid_label):
    train_set = lightgbm.Dataset(train.values, label=train_label)
    val_set = lightgbm.Dataset(valid.values, label=valid_label)
    
    model = lightgbm.train(train_set=train_set,
                           valid_sets=[val_set],
                           num_boost_round=3000,
                           params=params,
                           feval=lgb_f1_score,
                           early_stopping_rounds=100)
    return model
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)  # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [18]:
model=lgb_model_for_offline_test(train,train_label,valid,valid_label)

[1]	valid_0's binary_logloss: 0.410084	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.381341	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.35793	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.338231	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.320909	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.305632	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.291993	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.279816	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.269019	valid_0's f1: 0
[10]	valid_0's binary_logloss: 0.259194	valid_0's f1: 0.685714
[11]	valid_0's binary_logloss: 0.251878	valid_0's f1: 0.703107
[12]	valid_0's binary_logloss: 0.24318	valid_0's f1: 0.723669
[13]	valid_0's binary_logloss: 0.235203	valid_0's f1: 0.747228
[14]	valid_0's binary_logloss: 0.228024	valid_0's f1: 0.770234
[15]	valid_0's binary_logloss: 0.221222	valid_0's f1: 0.792353
[16]	valid_0's binary_logloss: 0.215086	valid_0's f1: 0.798732
[17]

[132]	valid_0's binary_logloss: 0.114239	valid_0's f1: 0.86673
[133]	valid_0's binary_logloss: 0.114155	valid_0's f1: 0.865094
[134]	valid_0's binary_logloss: 0.114176	valid_0's f1: 0.865911
[135]	valid_0's binary_logloss: 0.114168	valid_0's f1: 0.86632
[136]	valid_0's binary_logloss: 0.114066	valid_0's f1: 0.86673
[137]	valid_0's binary_logloss: 0.114045	valid_0's f1: 0.865784
[138]	valid_0's binary_logloss: 0.113995	valid_0's f1: 0.865248
[139]	valid_0's binary_logloss: 0.113909	valid_0's f1: 0.865784
[140]	valid_0's binary_logloss: 0.113819	valid_0's f1: 0.864839
[141]	valid_0's binary_logloss: 0.113848	valid_0's f1: 0.864151
[142]	valid_0's binary_logloss: 0.113799	valid_0's f1: 0.864407
[143]	valid_0's binary_logloss: 0.113826	valid_0's f1: 0.865475
[144]	valid_0's binary_logloss: 0.113776	valid_0's f1: 0.865221
[145]	valid_0's binary_logloss: 0.113703	valid_0's f1: 0.866416
[146]	valid_0's binary_logloss: 0.113769	valid_0's f1: 0.866416
[147]	valid_0's binary_logloss: 0.113741	va

In [20]:
pd.DataFrame(zip(list(train.columns),model.feature_importance())).sort_values(1,ascending=False)#.to_csv(r'feature_iportance_v2.csv')

,0,1
41,岗位编号平均被匹配数,387
1,求职者编号,349
33,求职者求职数,198
0,岗位编号,192
55,求职意向岗位类别平均被匹配数,184
...,...,...
36,工作年限是否符合,0
106,岗位编号_可到职天数_mode,0
105,岗位编号_可到职天数_median,0
102,岗位编号_可到职天数_max,0


# lgb_class

In [138]:
import lightgbm
from sklearn.model_selection import KFold
X = pd.concat([train,valid],ignore_index=True).reset_index(drop=True)
y = pd.concat([train_label,valid_label],ignore_index=True).reset_index(drop=True)
oof = []
prediction = test[['岗位编号', '求职者编号']]
prediction['pred'] = 0
df_importance_list = []
# model = lightgbm.LGBMClassifier(num_leaves=64,
#                            max_depth=10,
#                            learning_rate=0.1,
#                            n_estimators=1000000,
#                            subsample=0.8,
#                            feature_fraction=0.8,
#                            reg_alpha=0.5,
#                            reg_lambda=0.5,
#                            random_state=2048,
#                            metric='auc')
kfold = KFold(n_splits=5, random_state=2048, shuffle=True)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(X, y)):
    X_train = X.loc[trn_idx]
    Y_train = y.loc[trn_idx]

    X_val = X.loc[val_idx]
    Y_val = y.loc[val_idx]
    train_set = lightgbm.Dataset(X_train.values, label=Y_train)
    val_set = lightgbm.Dataset(X_val.values, label=Y_val)
    
    lgb_model = lightgbm.train(train_set=train_set,
                           valid_sets=[val_set],
                           num_boost_round=3000,
                           params=params,
                           feval=lgb_f1_score,
                           early_stopping_rounds=100)
    print('\nFold_{} Training ================================\n'.format(fold_id + 1))

    pred_val = lgb_model.predict(X_val)
    df_oof = X.iloc[val_idx][['岗位编号', '求职者编号']].copy()
    df_oof['标签'] = y.iloc[val_idx][['标签']].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict(test0[list(test0.columns)[:2]+list(test0.columns)[3:]].values)
    prediction['pred'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': list(test0.columns)[:2]+list(test0.columns)[3:],
        'importance': model.feature_importances_,
    })
    df_importance_list.append(df_importance)


d:\software\python\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


[1]	valid_0's binary_logloss: 0.410598	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.381586	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.357489	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.337372	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.320159	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.308309	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.294223	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.285244	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.27345	valid_0's f1: 0.00177936
[10]	valid_0's binary_logloss: 0.262659	valid_0's f1: 0.672963
[11]	valid_0's binary_logloss: 0.253074	valid_0's f1: 0.684611
[12]	valid_0's binary_logloss: 0.244214	valid_0's f1: 0.708046
[13]	valid_0's binary_logloss: 0.235963	valid_0's f1: 0.732807
[14]	valid_0's binary_logloss: 0.22846	valid_0's f1: 0.754008
[15]	valid_0's binary_logloss: 0.221472	valid_0's f1: 0.775974
[16]	valid_0's binary_logloss: 0.217058	valid_0's f1: 0.7

[137]	valid_0's binary_logloss: 0.112684	valid_0's f1: 0.861582
[138]	valid_0's binary_logloss: 0.112678	valid_0's f1: 0.86064
[139]	valid_0's binary_logloss: 0.112685	valid_0's f1: 0.86064
[140]	valid_0's binary_logloss: 0.112693	valid_0's f1: 0.860235
[141]	valid_0's binary_logloss: 0.112565	valid_0's f1: 0.859699
[142]	valid_0's binary_logloss: 0.11247	valid_0's f1: 0.859699
[143]	valid_0's binary_logloss: 0.112459	valid_0's f1: 0.859162
[144]	valid_0's binary_logloss: 0.112395	valid_0's f1: 0.859699
[145]	valid_0's binary_logloss: 0.112399	valid_0's f1: 0.859699
[146]	valid_0's binary_logloss: 0.112414	valid_0's f1: 0.860771
[147]	valid_0's binary_logloss: 0.112412	valid_0's f1: 0.860771
[148]	valid_0's binary_logloss: 0.112424	valid_0's f1: 0.860235
[149]	valid_0's binary_logloss: 0.112382	valid_0's f1: 0.860498
[150]	valid_0's binary_logloss: 0.112352	valid_0's f1: 0.860367
[151]	valid_0's binary_logloss: 0.112301	valid_0's f1: 0.860367
[152]	valid_0's binary_logloss: 0.112382	va

[272]	valid_0's binary_logloss: 0.111937	valid_0's f1: 0.868384
[273]	valid_0's binary_logloss: 0.111968	valid_0's f1: 0.868384
[274]	valid_0's binary_logloss: 0.111961	valid_0's f1: 0.867041
[275]	valid_0's binary_logloss: 0.111973	valid_0's f1: 0.867447
[276]	valid_0's binary_logloss: 0.112004	valid_0's f1: 0.867978
[277]	valid_0's binary_logloss: 0.112116	valid_0's f1: 0.867447
[278]	valid_0's binary_logloss: 0.112118	valid_0's f1: 0.867447
[279]	valid_0's binary_logloss: 0.112085	valid_0's f1: 0.867854
[280]	valid_0's binary_logloss: 0.112092	valid_0's f1: 0.867447
[281]	valid_0's binary_logloss: 0.11208	valid_0's f1: 0.866917
[282]	valid_0's binary_logloss: 0.112079	valid_0's f1: 0.867323
[283]	valid_0's binary_logloss: 0.112045	valid_0's f1: 0.867323
[284]	valid_0's binary_logloss: 0.112048	valid_0's f1: 0.866511
[285]	valid_0's binary_logloss: 0.112076	valid_0's f1: 0.866105
[286]	valid_0's binary_logloss: 0.112122	valid_0's f1: 0.8657
[287]	valid_0's binary_logloss: 0.112213	va

d:\software\python\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

[174]	valid_0's binary_logloss: 0.112203	valid_0's f1: 0.866254
[175]	valid_0's binary_logloss: 0.11224	valid_0's f1: 0.866254
[176]	valid_0's binary_logloss: 0.112334	valid_0's f1: 0.865842
[177]	valid_0's binary_logloss: 0.112267	valid_0's f1: 0.866254
[178]	valid_0's binary_logloss: 0.112269	valid_0's f1: 0.866794
[179]	valid_0's binary_logloss: 0.112289	valid_0's f1: 0.866254
[180]	valid_0's binary_logloss: 0.112284	valid_0's f1: 0.866254
[181]	valid_0's binary_logloss: 0.112238	valid_0's f1: 0.866794
[182]	valid_0's binary_logloss: 0.112267	valid_0's f1: 0.866794
[183]	valid_0's binary_logloss: 0.112314	valid_0's f1: 0.866381
[184]	valid_0's binary_logloss: 0.112344	valid_0's f1: 0.865842
[185]	valid_0's binary_logloss: 0.112394	valid_0's f1: 0.866254
[186]	valid_0's binary_logloss: 0.112414	valid_0's f1: 0.866254
[187]	valid_0's binary_logloss: 0.112397	valid_0's f1: 0.867332
[188]	valid_0's binary_logloss: 0.112379	valid_0's f1: 0.867584
[189]	valid_0's binary_logloss: 0.112446	

[43]	valid_0's binary_logloss: 0.135972	valid_0's f1: 0.851292
[44]	valid_0's binary_logloss: 0.134662	valid_0's f1: 0.852555
[45]	valid_0's binary_logloss: 0.133464	valid_0's f1: 0.85214
[46]	valid_0's binary_logloss: 0.13249	valid_0's f1: 0.851312
[47]	valid_0's binary_logloss: 0.13141	valid_0's f1: 0.853398
[48]	valid_0's binary_logloss: 0.130609	valid_0's f1: 0.851726
[49]	valid_0's binary_logloss: 0.129589	valid_0's f1: 0.854096
[50]	valid_0's binary_logloss: 0.128704	valid_0's f1: 0.853682
[51]	valid_0's binary_logloss: 0.127929	valid_0's f1: 0.853965
[52]	valid_0's binary_logloss: 0.127234	valid_0's f1: 0.852856
[53]	valid_0's binary_logloss: 0.126428	valid_0's f1: 0.852998
[54]	valid_0's binary_logloss: 0.125766	valid_0's f1: 0.854106
[55]	valid_0's binary_logloss: 0.125059	valid_0's f1: 0.855352
[56]	valid_0's binary_logloss: 0.124448	valid_0's f1: 0.856455
[57]	valid_0's binary_logloss: 0.123881	valid_0's f1: 0.856593
[58]	valid_0's binary_logloss: 0.123292	valid_0's f1: 0.85

[176]	valid_0's binary_logloss: 0.109957	valid_0's f1: 0.86553
[177]	valid_0's binary_logloss: 0.109945	valid_0's f1: 0.866888
[178]	valid_0's binary_logloss: 0.109879	valid_0's f1: 0.86594
[179]	valid_0's binary_logloss: 0.109854	valid_0's f1: 0.866477
[180]	valid_0's binary_logloss: 0.109874	valid_0's f1: 0.867299
[181]	valid_0's binary_logloss: 0.109885	valid_0's f1: 0.864455
[182]	valid_0's binary_logloss: 0.109838	valid_0's f1: 0.865813
[183]	valid_0's binary_logloss: 0.109829	valid_0's f1: 0.865813
[184]	valid_0's binary_logloss: 0.109806	valid_0's f1: 0.866761
[185]	valid_0's binary_logloss: 0.10976	valid_0's f1: 0.866888
[186]	valid_0's binary_logloss: 0.109688	valid_0's f1: 0.866351
[187]	valid_0's binary_logloss: 0.109707	valid_0's f1: 0.86594
[188]	valid_0's binary_logloss: 0.109719	valid_0's f1: 0.864865
[189]	valid_0's binary_logloss: 0.109683	valid_0's f1: 0.865403
[190]	valid_0's binary_logloss: 0.109713	valid_0's f1: 0.864455
[191]	valid_0's binary_logloss: 0.109595	val

[14]	valid_0's binary_logloss: 0.230825	valid_0's f1: 0.730769
[15]	valid_0's binary_logloss: 0.223949	valid_0's f1: 0.742441
[16]	valid_0's binary_logloss: 0.219227	valid_0's f1: 0.75
[17]	valid_0's binary_logloss: 0.213101	valid_0's f1: 0.767633
[18]	valid_0's binary_logloss: 0.207491	valid_0's f1: 0.787749
[19]	valid_0's binary_logloss: 0.202326	valid_0's f1: 0.80085
[20]	valid_0's binary_logloss: 0.197416	valid_0's f1: 0.806316
[21]	valid_0's binary_logloss: 0.194213	valid_0's f1: 0.807147
[22]	valid_0's binary_logloss: 0.18986	valid_0's f1: 0.813347
[23]	valid_0's binary_logloss: 0.185766	valid_0's f1: 0.817427
[24]	valid_0's binary_logloss: 0.181975	valid_0's f1: 0.821465
[25]	valid_0's binary_logloss: 0.178367	valid_0's f1: 0.825998
[26]	valid_0's binary_logloss: 0.174992	valid_0's f1: 0.829169
[27]	valid_0's binary_logloss: 0.17193	valid_0's f1: 0.831894
[28]	valid_0's binary_logloss: 0.168925	valid_0's f1: 0.835776
[29]	valid_0's binary_logloss: 0.166115	valid_0's f1: 0.837702

[144]	valid_0's binary_logloss: 0.115493	valid_0's f1: 0.859583
[145]	valid_0's binary_logloss: 0.11539	valid_0's f1: 0.860399
[146]	valid_0's binary_logloss: 0.115323	valid_0's f1: 0.862968
[147]	valid_0's binary_logloss: 0.11532	valid_0's f1: 0.86148
[148]	valid_0's binary_logloss: 0.115343	valid_0's f1: 0.86148
[149]	valid_0's binary_logloss: 0.115333	valid_0's f1: 0.860531
[150]	valid_0's binary_logloss: 0.115364	valid_0's f1: 0.860388
[151]	valid_0's binary_logloss: 0.115376	valid_0's f1: 0.860795
[152]	valid_0's binary_logloss: 0.115374	valid_0's f1: 0.860795
[153]	valid_0's binary_logloss: 0.115418	valid_0's f1: 0.861742
[154]	valid_0's binary_logloss: 0.115341	valid_0's f1: 0.860927
[155]	valid_0's binary_logloss: 0.115361	valid_0's f1: 0.86052
[156]	valid_0's binary_logloss: 0.115359	valid_0's f1: 0.860652
[157]	valid_0's binary_logloss: 0.115212	valid_0's f1: 0.862541
[158]	valid_0's binary_logloss: 0.115136	valid_0's f1: 0.862671
[159]	valid_0's binary_logloss: 0.115042	vali

In [140]:
from sklearn.metrics import roc_auc_score
df_oof = pd.concat(oof)
df_oof.sort_values(['pred'], inplace=True, ascending=False)
df_oof.reset_index(drop=True, inplace=True)
df_oof['pred_label'] = 0
df_oof.loc[:int(0.15 * len(df_oof)), 'pred_label'] = 1
f1 = f1_score(df_oof["标签"], df_oof['pred_label'])
auc = roc_auc_score(df_oof["标签"], df_oof['pred'])
f1, auc

(0.8648648648648649, 0.9789243440281573)

In [141]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby([
    'column'
])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,岗位编号平均被匹配数,416
1,求职者编号,414
2,自荐信字数,364
3,应聘者专业平均被匹配数,338
4,具体要求字数,314
5,求职意向岗位类别平均被匹配数,312
6,岗位编号,292
7,平均主要业绩字数,291
8,岗位求职者数,282
9,总主要业绩字数,237


In [142]:
prediction.sort_values(['pred'], inplace=True, ascending=False)
prediction.reset_index(drop=True, inplace=True)
prediction['LABEL'] = 0
prediction.loc[:int(0.16 * len(prediction)), 'LABEL'] = 1
prediction['LABEL'].value_counts()

d:\software\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
d:\software\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
d:\software\python\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

0    59450
1    11324
Name: LABEL, dtype: int64

In [146]:
prediction.columns=['RECRUIT_ID', 'PERSON_ID','PREDICT', 'LABEL']
name = "k_fold_55_feature"
prediction[['RECRUIT_ID', 'PERSON_ID', 'LABEL']].to_csv(f'../result/{name}.csv',
                                                        index=False)

In [18]:
pred_val

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [51]:
from utils import submission

In [17]:
path2=path0+'/result/'
name ='baseline_offline_2'
submission(path2,model,valid0,name)

..\utils.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["预测打分"] = model.predict(data[data.columns[:-3]].values)


## online

In [52]:
train_data.head()

,Unnamed: 0,岗位编号,求职者编号,标签
0,0,775040,5969825,0
1,1,815337,6212884,0
2,2,770107,5451198,0
3,3,43795661,320873919,1
4,4,798922,5650936,0


In [40]:

# online
data=pd.concat([train_data,valid_data],ignore_index=True)
train=get_k_fold_data(data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)

temp_data= pd.concat([train, test], ignore_index=True)
# data_all_feature=feature_process(temp_data)
temp_data=deliver_record_feature(temp_data)
for f in temp_data.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data[f] = le.fit_transform(temp_data[f].astype('str'))
temp_data["招聘对象"] =le.fit_transform(temp_data["招聘对象"].astype('str'))
temp_data["最近工作岗位"] =le.fit_transform(temp_data["最近工作岗位"].astype('str'))
temp_data["求职意向岗位类别"] =le.fit_transform(temp_data["求职意向岗位类别"].astype('str'))





应聘者专业_招聘职位_simhash，共耗时: 13.27483582496643
应聘者专业_对应聘者的专业要求_simhash，共耗时: 15.615927457809448
应聘者专业_具体要求_simhash，共耗时: 30.477040767669678
最近工作岗位_招聘职位_simhash，共耗时: 15.780963897705078
最近工作岗位_对应聘者的专业要求_simhash，共耗时: 16.204984664916992
最近工作岗位_具体要求_simhash，共耗时: 27.978307962417603
最近所在行业_招聘职位_simhash，共耗时: 16.0258207321167
最近所在行业_对应聘者的专业要求_simhash，共耗时: 15.419012308120728
最近所在行业_具体要求_simhash，共耗时: 27.191560745239258
专业特长_招聘职位_simhash，共耗时: 26.9104106426239
专业特长_对应聘者的专业要求_simhash，共耗时: 27.132718801498413
专业特长_具体要求_simhash，共耗时: 38.82530117034912
求职意向岗位类别_招聘职位_simhash，共耗时: 16.006715297698975
求职意向岗位类别_对应聘者的专业要求_simhash，共耗时: 18.383556365966797
求职意向岗位类别_具体要求_simhash，共耗时: 29.482659339904785
求职意向所在行业_招聘职位_simhash，共耗时: 16.839683055877686
求职意向所在行业_对应聘者的专业要求_simhash，共耗时: 16.52983546257019
求职意向所在行业_具体要求_simhash，共耗时: 25.28001880645752
自荐信_招聘职位_simhash，共耗时: 32.05219340324402
自荐信_对应聘者的专业要求_simhash，共耗时: 29.798278331756592
自荐信_具体要求_simhash，共耗时: 38.446274280548096
应聘者专业_招聘职位_sim_jaccard，共耗时: 7.057034015655518
应聘者专业_对应聘者

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:14<00:00,  2.07s/it]


应聘者专业
最近工作岗位
最近所在行业
当前工作所在地
语言能力
专业特长
招聘职位
对应聘者的专业要求
岗位工作地点
具体要求
自荐信
求职意向工作地点
求职意向所在行业
其他说明
汉语语言能力
英语语言能力
粤语语言能力
日语语言能力


KeyError: '岗位类别'

In [54]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,自荐信字数,可到职天数,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,50.0,7.0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,148.0,7.0,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,65.0,7.0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,39.0,7.0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,NaN,7.0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [55]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [42]:
def lgb_model_for_upload(data, label):
    model = lightgbm.train(train_set=lightgbm.Dataset(data.values, label.values),
                           num_boost_round=800,
                           params=params)

    return model

In [43]:
data_all_feature  =temp_data
#train,train_label,valid,valid_label
train0 = data_all_feature[:train.shape[0]]#.标签.value_counts()
test0 = data_all_feature[train.shape[0]:]
train_label = train0["标签"]
train=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]


In [44]:
model = lgb_model_for_upload(train, train_label)

In [88]:
# from utils import submission
def submission(path, model, test0, name):
    test0["预测打分"] = model.predict(test0[list(test0.columns)[:2]+list(test0.columns)[3:]].values)
    test_data = test0.sort_values("预测打分", ascending=False, ignore_index=True)
    test_data["预测"] = 0
    test_data.loc[:int(0.15 * len(test_data)), ["预测"]] = 1

    submit = test_data[["岗位编号", "求职者编号", "预测"]]
    submit.columns = ["RECRUIT_ID", "PERSON_ID", "LABEL"]
    submit.to_csv(path + name + ".csv", index=False)
path2=path0+'/result/'
name ='online_add_person_match_feature'
submission(path2,model,test0,name)

In [89]:
test0=test0[test0.columns[:-1]]

In [86]:
test0

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate,预测打分
0,135144,996499,0,1,15,1.0,80,34,226,0,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.943105
1,135144,3301030,0,1,12,2.0,112,37,287,0,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.989315
2,135144,3404860,0,1,0,2.0,0,32,1,60,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.714051
3,135144,4155520,0,1,11,3.0,0,32,1,60,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.859480
4,135144,4211720,0,1,12,1.0,211,30,226,0,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,1.0,0.95,0.602499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70769,44350657,321173912,0,1,5,4.0,377,26,198,38,...,7.0,56.30374,7.0,7.0,0.555556,1,0.888889,1.0,1.00,0.999980
70770,44350657,321377962,0,1,0,4.0,292,28,226,0,...,7.0,56.30374,7.0,7.0,0.555556,1,0.888889,1.0,1.00,0.999992
70771,47578782,319583916,0,0,0,4.0,302,28,226,0,...,7.0,NaN,7.0,7.0,1.000000,0,1.000000,0.0,1.00,0.999959
70772,47649893,6079047,0,1,7,4.0,0,29,174,82,...,7.0,0.00000,7.0,7.0,1.000000,1,1.000000,0.0,1.00,0.997052


In [78]:
path=path0+'/result/'
name ='online_add_person_match_feature_v2'
test0 = test0.reset_index(drop=True)
test_data["标签"] =test0["预测打分"].apply(lambda x :1 if x>=0.5 else 0)

submit = test_data[["岗位编号", "求职者编号", "标签"]]
submit.columns = ["RECRUIT_ID", "PERSON_ID", "LABEL"]
submit.to_csv(path + name + ".csv", index=False)


In [81]:
train0["标签"].value_counts()

0    29670
1     5621
Name: 标签, dtype: int64

In [83]:
5621/(29670+5621)

0.15927573602334874

In [56]:
temp_result=test0.sort_values("预测打分", ascending=False, ignore_index=True)
temp_result[temp_result.预测打分<0.5].shape,temp_result[temp_result.预测打分>=0.5].shape

((60855, 177), (9919, 177))

In [51]:
train0["标签"].value_counts()

0    29670
1     5621
Name: 标签, dtype: int64

In [52]:
5621/(29670+5621)

0.15927573602334874

In [55]:
9919/(60855+9919)

0.14015033769463361

In [21]:
# baseline offline: 0.8521
# online  ：0.8544

In [78]:
import numpy as np
ls=[np.array([1,2,3]),np.array([4,5,6])]

In [83]:
sum(ls)

array([5, 7, 9])

* one-hot +岗位编号	求职者编号 0.8633      ONLINE:0.8653
* no one hot +岗位编号	求职者编号 0.865602      ONLINE:0.8663
* 添加招聘岗位投递记录中的求职意向岗位，求职者专业，（年龄、工作经历数、项目经验数、可到职天数）统计指标、工作地点符合占比特征
* 0.8665 online：0.86748
* 添加k-折交叉的标签特征：offline:0.8772  online: 0.8711

In [2]:
import pandas as pd
df1 = pd.DataFrame([1,2,3,4,5,6])

In [5]:
df1[1]=[1,1,1,1,2,2]

In [6]:
df1

,0,1
0,1,1
1,2,1
2,3,1
3,4,1
4,5,2
5,6,2


In [7]:
f1=1
f2=0
df1.groupby(f1)[f2].agg(
        [(f"{f1}_{f2}_mean", "mean"),
         (f"{f1}_{f2}_max", "max"),
         (f"{f1}_{f2}_min", "min"),
         (f"{f1}_{f2}_std", "std")]
    ).reset_index()

,1,1_0_mean,1_0_max,1_0_min,1_0_std
0,1,2.5,4,1,1.290994
1,2,5.5,6,5,0.707107
